# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [45]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [46]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [47]:
### Dropping missing values
spaceship.dropna(inplace=True)

### Drop unnecessary columns
spaceship.drop(columns=['Name', 'PassengerId'], inplace=True)

### Transforming 'Cabin' to just the first letter
spaceship['Cabin'] = spaceship['Cabin'].str[0]

# Label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
spaceship['HomePlanet'] = le.fit_transform(spaceship['HomePlanet'])
spaceship['CryoSleep'] = le.fit_transform(spaceship['CryoSleep'])
spaceship['Cabin'] = le.fit_transform(spaceship['Cabin'])
spaceship['Destination'] = le.fit_transform(spaceship['Destination'])
spaceship['VIP'] = le.fit_transform(spaceship['VIP'])

spaceship.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,1,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False
1,0,0,5,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True
2,1,0,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False
3,1,0,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False
4,0,0,5,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True


In [48]:
### Splitting the data into features and target

features = spaceship.drop(columns = ["Transported", "FoodCourt", "Spa", "ShoppingMall", "VIP", "Destination"])
target = spaceship["Transported"].astype(int)

**Perform Train Test Split**

In [49]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=42)

In [50]:
### Normalizing and transforming the data

from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()
normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [51]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score


In [52]:
### Bagging Classifier
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth = 8),
                              n_estimators= 200,
                              max_samples = 3000,
                              bootstrap=True)

In [53]:
bagging.fit(X_train_norm, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=8),
                  max_samples=3000, n_estimators=200)

In [54]:
### Accuracy score

pred = bagging.predict(X_train_norm)
print("Train Accuracy:", accuracy_score(y_train, pred))

pred = bagging.predict(X_test_norm)
print("Test Accuracy:", accuracy_score(y_test, pred))

Train Accuracy: 0.7734670704012112
Test Accuracy: 0.7503782148260212


In [55]:
### Pasting Classifier
pasting = BaggingClassifier(DecisionTreeClassifier(max_depth = 8),
                              n_estimators= 200,
                              max_samples = 3000,
                              bootstrap=False)

In [56]:
pasting.fit(X_train_norm, y_train)

BaggingClassifier(bootstrap=False,
                  estimator=DecisionTreeClassifier(max_depth=8),
                  max_samples=3000, n_estimators=200)

In [57]:
pasting.fit(X_train_norm, y_train)

BaggingClassifier(bootstrap=False,
                  estimator=DecisionTreeClassifier(max_depth=8),
                  max_samples=3000, n_estimators=200)

In [58]:
### Accuracy score

pred = pasting.predict(X_train_norm)
print("Train Accuracy:", accuracy_score(y_train, pred))

pred = pasting.predict(X_test_norm)
print("Test Accuracy:", accuracy_score(y_test, pred))

Train Accuracy: 0.7757380772142316
Test Accuracy: 0.7465960665658093


- Random Forests

In [59]:
forest = RandomForestClassifier()
forest.fit(X_train_norm, y_train)

RandomForestClassifier()

In [60]:
### Accuracy score

pred = forest.predict(X_train_norm)
print("Train Accuracy:", accuracy_score(y_train, pred))

pred = forest.predict(X_test_norm)
print("Test Accuracy:", accuracy_score(y_test, pred))

Train Accuracy: 0.9163512490537472
Test Accuracy: 0.7420574886535553


- Gradient Boosting

In [61]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

In [62]:
gradient = GradientBoostingClassifier(max_depth=10, n_estimators=500, learning_rate=0.05)
gradient.fit(X_train_norm, y_train)

GradientBoostingClassifier(learning_rate=0.05, max_depth=10, n_estimators=500)

In [63]:
### Accuracy score

pred = gradient.predict(X_train_norm)
print("Train Accuracy:", accuracy_score(y_train, pred))

pred = gradient.predict(X_test_norm)
print("Test Accuracy:", accuracy_score(y_test, pred))

Train Accuracy: 0.9161619984859954
Test Accuracy: 0.7329803328290468


- Adaptive Boosting

In [64]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10),
                         n_estimators=100,
                         learning_rate=0.5)
ada.fit(X_train_norm, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10),
                   learning_rate=0.5, n_estimators=100)

In [65]:
### Accuracy score

pred = ada.predict(X_train_norm)
print("Train Accuracy:", accuracy_score(y_train, pred))

pred = ada.predict(X_test_norm)
print("Test Accuracy:", accuracy_score(y_test, pred))

Train Accuracy: 0.8330809992429977
Test Accuracy: 0.7428139183055976


Which model is the best and why?

In [66]:
#comment here